# Keyword Extraction using 'soykeyword'

### 설명
- 한 문서 집합의 키워드 :
    - 다른 문서 집합과 해당 문서 집합을 구분할 수 있는 질 좋은 단어(구분력, discriminative power)
    - 해당 집합을 잘 설명할 수 있는 (설명력, high coverage) 단어
    - 빈도수가 낮은 단어는 한 집합에서만 등장할 가능성이 높기 때문에 구분력은 크지만 설명력이 약합니다
    - 제안된 두 가지 알고리즘은 높은 설명력과 구분력을 동시에 지니는 단어들을 키워드로 선택합니다.
<br><br>
- 연관어 :
    - 기준 단어가 포함된 문서 집합과 포함되지 않은 문서 집합을 구분하는 키워드를 연관어로 정의
    - co-occurrence 가 높은 단어라는 의미이기도 합니다
    - co-occurrence 가 높으면서도 설명력이 좋은 단어를 선택합니다.

Setup

In [11]:
!pip install psutil
!pip install soykeyword

Requires

In [6]:
import sys
import numpy as np
import sklearn
import psutil

print('python version ', sys.version) # >=3.4
print('numpy version ', np.__version__) # >= 1.12.1
print('sklearn version ', sklearn.__version__) # >= 0.18
print("psutil version ", psutil.__version__) # >= 5.0.1

python version  3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]
numpy version  1.21.0
sklearn version  1.0.2
psutil version  5.9.0


In [107]:
import pandas as pd
from eunjeon import Mecab
from tqdm import tqdm

### Lasso Regression Keyword Extractor
- sparse matrix x를 extractor에 입력
- index2word : word idx에 대한 단어 list 형식

In [12]:
from soykeyword.lasso import LassoKeywordExtractor

lassobased_extractor = LassoKeywordExtractor(min_tf=20, min_df=10)
lassobased_extractor.train(x, index2word)

In [ ]:
keywords = lassobased_extractor.extract_from_docs(
    documents, 
    min_num_of_keywords=30
)

In [ ]:
lassobased_extractor.extract_from_word(
    '아이오아이',
    min_num_of_keywords=30
)

### Keyword extraction using Proportion Ratio

In [31]:
df = pd.read_csv('../data/train.csv')
df.head()

,docID,date,writerName,title,content,tag
0,1,20230214,머니S,"정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원",정부가 미국 AI 개발업체인 '오픈AI'(OpenAI)의 '챗GPT'(ChatGPT...,0
1,2,20230215,뉴시스,인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각,"기사내용 요약 법원 ""피의 사실 일부 다툼 여지, 불구속 상태 방어권 보장 필요"" ...",0
2,3,20230214,아이뉴스24,튀르키예 강진에 우리나라 지하수가 출렁였다,튀르키예에서 발생한 강진에 우리나라의 지하수가 출렁였다는 관측 보고가 나왔다. 한국...,0
3,4,20230215,데일리안,"멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행",[데일리안 = 박영민 기자] 오픈마켓 멸치쇼핑이 2023년 신입 및 경력 사원을 대...,0
4,5,20230111,뉴스1,"美국방부, 추모의 벽 전사자 명단 오류에 ""유감스러운 실수""","국방부 대변인 ""실수 바로잡기 위해 내무부와 협력""…'오류 발견' 가족 등에 연락 ...",1


Proportion ratio를 이용하는 키워드 추출 방법은 CorpusbasedKeywordExtractor와 MatrixbasedKeywordExtractor 두 종류로 구현되어 있습니다. CorpusbasedKeywordExtractor는 아래 예시처럼 text 파일에서 키워드를 직접 추출하는 코드이며, MatrixbasedKeywordExtractor는 sparse format으로 만들어둔 term frequency matrix에서 키워드를 추출하는 코드입니다.

Proportion ratio의 개념은 아래와 같습니다.

키워드란 사실 명확한 정의가 있는 단어가 아닙니다. 흔히 사용하는 키워드의 정의에는 자주 나오는 단어나, TF-IDF 값이 있습니다. 이들은 각자의 관점으로 키워드를 정의한 것입니다. 자주 나오는 단어를 키워드로 정의하는 것은 많이 나올수록 키워드라는 의미이며, 이 때에는 조사와 같은 단어가 키워드가 될 수도 있습니다. 이를 보완하기 위해 품사 판별 (Part-of-Speech tagging)을 한 뒤, 명사만을 추출하여 최빈어를 키워드로 선정하는 것은 합리적이라 생각됩니다. TF-IDF를 키워드로 사용하는 방법은 조금 위험한 방법입니다. IDF는 단어가 등장한 문서의 개수가 적을수록 커지기 때문에 오히려 노이즈일 가능성이 높기 때문입니다.

그 외에도 chi-square를 이용하는 방법도 있습니다. 제가 제안하는 Proportion ratio는 이 방법에 가깝습니다. 기본 컨셉은 아래와 같습니다. '뉴스에서의 키워드'를 선택하라는 말은 애매모호합니다. 하지만, '오늘의 뉴스에서의 키워드'나 '아이오아이에 대한 문서에서의 키워드'라는 말은 조금 더 명료합니다. 키워드에 대한 관점이 생기기 때문입니다. 좀 더 자세히, 여름철 뉴스에서는 '호우'라는 단어가 0.1% 씩 늘 등장한다고 가정합시다. 어느날 '호우'라는 단어가 평상시와 다르게 0.9% 등장하였다면 (평상시보다 9배), 이 날은 정말로 호우가 내려서 뉴스에 그 단어가 자주 등장했을 가능성이 높습니다. 그렇다면 '호우'는 그날의 키워드가 될 수 있을 것입니다. 이를 수치로 만들기 위해서 다음과 같은 지표를 만들었습니다.

score(w) = P(w|Dt) / { P(w|Dt) + P(w|Dr) }

P(w|Dt): target document에서 단어 w가 출현한 비율
P(w|Dr): reference document에서 단어 w가 출현한 비율

Target document란 키워드를 정의하고 싶은 문서 집합을 의미합니다. '아이오아이라는 단어가 포함된 뉴스'라던가, 어느날의 뉴스가 됩니다. Reference document는 평상시의 문서 집합입니다. '아이오아이'를 포함한 연예뉴스 가 될 수도, 하루치 전체 뉴스가 될 수도 있습니다. 어느 하루의 뉴스의 키워드를 선택하기 위해서는 이전 10일치의 뉴스를 reference document로 선택할 수 있습니다.

이렇게 keyword score를 정의하면 score(w)는 [0, 1] 사이의 값이 됩니다. 평상시 호우가 0.1% 등장하였다가 오늘 0.9% 등장하였다면 score는 0.9 / (0.1 + 0.9) = 0.9 입니다. 평상시와 같이 0.1% 등장하였다면 (0.1 / (0.1 + 0.1)) = 0.5가 됩니다. 0.5란 평상시와 다르지 않다는 의미이며, 그 이하는 평상시보다 등장하지 않았다는 의미입니다. 하지만 0.5보다 작은 값은 의미가 없습니다. target document set은 reference document set보다 훨씬 작은 집합이기 때문에 많은 단어를 포함하지 않을 수 있기 때문입니다. 대신 0.5보다 큰 score를 지니는 단어들은 평상시보다 자주 등장한 단어임을 의미합니다. 이때에는 한가지 false alarm이 생길 수 있습니다. 애초에 자주 등장하지 않는 단어이기 때문에 target documents에만 등장하는 단어는 1.0에 가까운 score를 가지게 됩니다. 이를 방지하기 위해서 최소한 등장해야 하는 단어 빈도수를 한정할 필요가 있습니다. 그래서 키워드를 선택할 때 항상 parameter로 min_frequency를 넣도록 하였습니다.

keywords = corpusbased_extractor.extract_from_word('아이오아이', min_score=0.8, min_frequency=100)

In [34]:
df[df['content'].isna()]

,docID,date,writerName,title,content,tag
132,133,20230105,뉴스1,"[속보]軍 ""북한 무인기, 용산 비행금지구역 북쪽 끝 일부 지나가""",NaN,1
178,179,20230104,뉴스1,"[속보]軍 ""합동 드론사령부 조기 창설… 드론킬러 드론 신속 개발""",NaN,1
216,217,20230212,코리아중앙데일리,Greek epics,NaN,1
385,386,20230214,뉴스1,"[속보] 한미일 ""北과 대화에 열려 있어…비핵화 대화 복귀 촉구""",NaN,1
399,400,20230203,연합뉴스,"[속보] 美 F-22·F-35B, 韓 F-35A와 서해상에서 또 연합공중훈련",NaN,1
...,...,...,...,...,...,...
7787,7788,20230214,중앙일보,[오늘의 날씨] 2월 14일,NaN,0
7836,7837,20230113,연합뉴스,[속보] 한일 외교장관 통화…강제징용 문제 의견교환,NaN,1
7854,7855,20230209,아시아경제,"[속보]北 ""전술핵운용부대 열병식 등장""…김주애 참석",NaN,1
7959,7960,20230131,연합뉴스,"[속보] 美 국방장관 ""앞으로 F-22·F-35·항모 전개 많이 할것""",NaN,1


In [35]:
df_dropna = df.dropna(axis=0, subset=['content']) # content가 없는 기사는 drop --> 총 102개
df_dropna['title_content'] = df_dropna['title'] + ' ' + df_dropna['content'] # title + content
df_dropna['title_content'].head()

C:\Users\vaiv\AppData\Local\Temp\ipykernel_7660\2103349724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna['title_content'] = df_dropna['title'] + ' ' + df_dropna['content'] # title + content


0    정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원 정부가 미...
1    인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각 기사내용 요약 법원 "피의 사...
2    튀르키예 강진에 우리나라 지하수가 출렁였다 튀르키예에서 발생한 강진에 우리나라의 지...
3    멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행 [데일리안 = 박영민 기...
4    美국방부, 추모의 벽 전사자 명단 오류에 "유감스러운 실수" 국방부 대변인 "실수 ...
Name: title_content, dtype: object

In [37]:
df_dropna[df_dropna['title_content'].isna()]

,docID,date,writerName,title,content,tag,title_content


In [45]:
# 명사 추출
mecab = Mecab()

# Get nouns from df['title_content']
tokenized_corpus_fname = df_dropna['title_content'].apply(lambda x: ' '.join(mecab.nouns(x)))

In [47]:
tokenized_corpus_fname

0       정부 반도체 석 박사 집중 육성 대학당 년 간 억 원 지원 정부 미국 개발 업체 오...
1       인사 청탁 대가 금품 수수 의혹 전 소방 청장 영장 기각 기사 내용 요약 법원 피 ...
2       튀르 키 예 강진 우리 나라 지하수 튀르 키 예 발생 강진 우리 나라 지하수 관측 ...
3       멸치 쇼핑 년 신입 경력 사원 규모 공채 진행 데일리안 박영민 기자 오픈 마켓 멸치...
4       美 국방부 추모 벽 전사자 명단 오류 유감 실수 국방부 대변인 실수 내무부 협력 오...
                              ...                        
7995    외교부 일 日 강제 징용 해법 토론회 국회 일 의원 연맹 공동 개최 데일리 권오석 ...
7996    교원 단체 유치원 명칭 일제 잔재 유아 학교 변경 교육부 년 유치원 어린이집 하나 ...
7997    일 외교 차관 회담 강제 동원 해법 이견 일 워싱턴 미일 차관 협의회 계기 양자 회...
7998    최상호 국립오페라단 단장 문화 체육관 광부 재단법인 국립오페라단 단장 겸 예술 감독...
7999    이통 통신 사 갤럭시 시리즈 사전 개통 시작 구매 팁 스포츠서울 황철 훈 기자 통신...
Name: title_content, Length: 7898, dtype: object

In [48]:
tokenized_corpus_fname[0].split(' ')

['정부',
 '반도체',
 '석',
 '박사',
 '집중',
 '육성',
 '대학당',
 '년',
 '간',
 '억',
 '원',
 '지원',
 '정부',
 '미국',
 '개발',
 '업체',
 '오픈',
 '챗',
 '등장',
 '산업',
 '시장',
 '격변',
 '반도체',
 '산업',
 '고급',
 '재양',
 '대학원',
 '사업',
 '공고',
 '시행',
 '과학',
 '기술',
 '정보',
 '통신부',
 '다음',
 '달',
 '일',
 '세계',
 '전문',
 '인력',
 '양성',
 '대학원',
 '신설',
 '규모',
 '예산',
 '지원',
 '일',
 '정부',
 '개',
 '대학원',
 '선정',
 '년',
 '년',
 '동안',
 '대학당',
 '억',
 '원',
 '상당',
 '지원',
 '방침',
 '정부',
 '이번',
 '사업',
 '국가',
 '전략',
 '기술',
 '경제',
 '안보',
 '핵심',
 '품목',
 '반도체',
 '분야',
 '기술',
 '경쟁력',
 '미래',
 '유망',
 '시장',
 '창출',
 '세계',
 '수준',
 '반도체',
 '설계',
 '소프트웨어',
 '전문',
 '고급',
 '인재',
 '명',
 '양성',
 '목표',
 '신입',
 '교육',
 '가을',
 '학기',
 '시작',
 '대학',
 '정부',
 '지원',
 '반도체',
 '설계',
 '소프트웨어',
 '역량',
 '확보',
 '가능',
 '특화',
 '교육',
 '과정',
 '구성',
 '시스템',
 '소프트웨어',
 '관련',
 '전문가',
 '등',
 '우수',
 '연구',
 '진',
 '확보',
 '전임',
 '교원',
 '인',
 '이상',
 '반도체',
 '제작',
 '경험',
 '등',
 '실전',
 '역량',
 '재고',
 '기업',
 '참여',
 '프로젝트',
 '등',
 '교과목',
 '구성',
 '이',
 '연계',
 '기업',
 '인턴',
 '십',
 '팹',
 '리스',
 '설계',
 '개발',
 '

In [60]:
class Corpus:
    def __init__(self, df):
        self.title = df['title']
        self.content = df['content']
        self.title_content = df['title_content']
        self.length = 0
    def __len__(self):
        if self.length == 0:
            self.length = len(df)
        return self.length
    def __iter__(self):
        for text in self.title_content:
            yield text.strip()

In [61]:
for i, doc in enumerate(Corpus(df_dropna)):
    if i <= 5: continue
    if i > 10: break
    print('doc=%d, num words=%d, %s\n' % (i, len(doc.split(' ')), doc[:200]))

doc=6, num words=35, 이라크 무역부 장관과 면담하는 원희룡 장관 (서울=연합뉴스) 원희룡 국토교통부 장관이 지난 25일 이라크 바그다드를 방문, 아티르 알 그레이리(Atheer Dawoud Salman Al Ghrairy) 이라크 무역부 장관과 면담하고 있다. 2023.1.27 [국토교통부 제공. 재판매 및 DB 금지] photo@yna.co.kr

doc=7, num words=117, 내달 ‘윤 2월’ 서울시립 화장터 2배로 운영 ‘개장 화장’ 22일부터 예약 서울시설공단은 3년 만의 윤달을 앞두고 서울시립 장사시설의 하루 운영 화구 수를 2배로 확대한다고 15일 밝혔다. 오는 3월22일부터 4월19일까지는 3년 만에 돌아오는 윤달이다. ‘손 없는 달’이라고도 불리는 이 기간에는 개장유골 화장 예약 건수가 평소보다 많이 늘어나는 경향이 

doc=8, num words=82, BTS 지민, 튀르키예·시리아 지진 피해 어린이 긴급구호 동참 [파이낸셜뉴스] 그룹 방탄소년단(BTS)의 지민이 ‘튀르키예·시리아 지진 피해 어린이 긴급구호’에 동참했다. 15일 유니세프한국위원회에 따르면 이날 방탄소년단의 지민이 ‘튀르키예·시리아 지진 피해 어린이 긴급구호’에 동참하며 1억원을 기부했다. 지민은 국내외 이슈마다 기금을 전하며 소외계층을 지

doc=9, num words=240, 창원서 창녕·밀양 접근성 높일 창원 동읍~봉강 도로 10.1㎞ 전 구간 개통 15일 동읍주민센터 앞 용잠교차로 상부서 준공식 개최 2173억 원 투입 2008년 착공해 15년 만에 전체 준공 교통량 분산 주민 통행·주남저수지 방문객 편의 기대 경남 창원 도심에서 주남저수지 일대는 물론 창녕과 밀양으로 가는 접근성 높일 동읍~봉강 도로가 완전히 개통했다. 경

doc=10, num words=75, 북한 해킹 조직 ‘김수키’, 방송사·기업도 표적 북한 해커조직 '김수키'가 만든 악성 코드가 방송사와 일반 기업까지 공격 표적으로 삼은 것으로 확인됐습니다. 정보 보안 기업 '안랩'

CorpusbasedKeywordExtractor를 만들 때, 애초에 키워드 후보가 될 수 있는 단어를 minimum term frequency (min_tf)와 minimum document frequency (min_df)로 필터링 할 수 있도록 하였습니다. 키워드의 후보들은 모두 min_tf, min_df 이상이 되는 단어들로 한정됩니다.

tokenize는 텍스트 형식의 corpus에서 단어를 추출하기 위한 tokenizer입니다. 기본값은 띄어쓰기입니다만, KoNLPy의 nouns()나 pos()를 이용할 수도 있습니다.

In [78]:
from soykeyword.proportion import CorpusbasedKeywordExtractor

mecab = Mecab()
corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    tokenize=lambda x:mecab.nouns(x),
    verbose=True
)

corpusbased_extractor.train(Corpus(df_dropna))

training was done 25128 terms, 7898 docs, memory = 0.738 Gb


In [63]:
for word in ['박근혜', '문재인', '최순실', '아이오아이', '트와이스', '군사', '외교']:
    print(word, corpusbased_extractor.frequency(word))

박근혜 55
문재인 192
최순실 0
아이오아이 0
트와이스 0
군사 1251
외교 2138


In [65]:
# '외교'가 포함된 문서 번호 (텍스트 파일에서의 line number)를 가지고 올 수도 있습니다.
documents = corpusbased_extractor.get_document_index('외교')
documents[:10]

[5, 10, 34, 35, 58, 61, 62, 76, 78, 83]

extract_from_word(aspect_word)는 기준점이 되는 단어를 넣으면 min_score, min_frequency 이상이 되는 단어들 을 선택합니다. target document는 aspect_word가 포함된 문서 집합이며, reference document는 aspect_word가 포함되지 않은 문서 집합입니다.

'외교'가 포함된 문서가 target document이기 때문에 '외교' 단어는 score가 반드시 1.0입니다. 그 외의 키워드에서 '뮌헨', '링컨', '방미', '외무', '조현동', '이란' 등 추출

In [72]:
keywords = corpusbased_extractor.extract_from_word(
    '외교',
    min_score=0.7,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word='외교', frequency=2138, score=1.0),
 KeywordScore(word='뮌헨', frequency=109, score=0.9870338069501552),
 KeywordScore(word='링컨', frequency=123, score=0.9844655198967406),
 KeywordScore(word='방미', frequency=194, score=0.9816726832390705),
 KeywordScore(word='외무', frequency=142, score=0.9766310794660197),
 KeywordScore(word='조현동', frequency=156, score=0.9677452793849101),
 KeywordScore(word='이란', frequency=448, score=0.9664862141387943),
 KeywordScore(word='한중', frequency=121, score=0.9651446180085678),
 KeywordScore(word='고위급', frequency=202, score=0.9630989958952829),
 KeywordScore(word='이사국', frequency=184, score=0.9548784756254747)]

In [67]:
for word in ['정치', '외교', '국방']:

    keywords = corpusbased_extractor.extract_from_word(
        word, min_score=0.8,
        min_frequency=150
    )

    keywords = keywords[:48]

    word_frequency = corpusbased_extractor.frequency(word)
    print('Aspect word = %s (%d)' % (word, word_frequency))

    def in_a_line(subkeywords):
        def tuple_to_strf(keyword):
            return '%s (%d, %.2f)' % keyword        
        strf = [tuple_to_strf(keyword) for keyword in subkeywords]
        strf = ['%17s' % s for s in strf]
        return '  -  '.join(strf)

    for i in range(12):
        subkeywords = keywords[4*i:4*i+4]
        line = in_a_line(subkeywords)
        print(line)

    print('-'*80)

Aspect word = 정치 (668)
   정치 (668, 1.00)  -      당 (178, 0.92)  -    김주애 (388, 0.92)  -     공직 (177, 0.89)
   선거 (307, 0.86)  -     대만 (512, 0.86)  -    인민군 (243, 0.85)  -     건군 (164, 0.85)
   행보 (169, 0.84)  -    김정은 (584, 0.84)  -     자제 (162, 0.84)  -     혁명 (172, 0.84)
   야당 (182, 0.84)  -     직무 (191, 0.84)  -     국정 (205, 0.83)  -    노동당 (166, 0.83)
   전원 (234, 0.83)  -     개입 (165, 0.83)  -    청와대 (233, 0.83)  -     신문 (342, 0.82)
   무력 (315, 0.82)  -    김일성 (190, 0.82)  -    의원 (1589, 0.82)  -     갈등 (314, 0.81)
   대리 (161, 0.81)  -     소아 (190, 0.81)  -    민주당 (628, 0.81)  -      절 (247, 0.81)
   여론 (248, 0.81)  -     통일 (311, 0.80)




--------------------------------------------------------------------------------
Aspect word = 외교 (2138)
  외교 (2138, 1.00)  -     방미 (194, 0.98)  -    조현동 (156, 0.97)  -     이란 (448, 0.97)
  고위급 (202, 0.96)  -    이사국 (184, 0.95)  -     국빈 (229, 0.95)  -     징용 (614, 0.94)
   해법 (544, 0.94)  -     박진 (531, 0.93)  -    안보리 (311, 0.93)  -    워싱턴 

다른 머신러닝 알고리즘에 적용하기 위해서 sparse matrix 형식으로 데이터를 저장해둔 경우들도 있습니다. 이때에도 키워드 추출이 용이하도록 MatrixbasedKeywordExtractor를 만들어두었습니다. Interface나 작동 방식은 위와 동일합니다.

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0.001)
x = vectorizer.fit_transform(Corpus(df_dropna))

print(x.shape)

(7898, 26969)


scikit-learn의 CountVectorizer를 이용하여 term frequency matrix; x를 만들고, {word:index}의 dictionary와 [word, ...]의 list of str을 만들어두었습니다.

In [70]:
word2index = vectorizer.vocabulary_
index2word = sorted(
    vectorizer.vocabulary_,
    key=lambda x:vectorizer.vocabulary_[x]
)

Matrix 형식이기 때문에 tokenize는 필요없습니다. 그 외의 parameters는 동일합니다.

In [74]:
from soykeyword.proportion import MatrixbasedKeywordExtractor

matrixbased_extractor = MatrixbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    verbose=True
)

matrixbased_extractor.train(x, index2word=None)

MatrixbasedKeywordExtractor trained


In [75]:
keywords = matrixbased_extractor.extract_from_word(
    5537,
    min_score=0.8,
    min_frequency=100
)

keywords[:10]

[KeywordScore(word=25560, frequency=101, score=0.963730934184232),
 KeywordScore(word=26773, frequency=108, score=0.9576198260296309),
 KeywordScore(word=6420, frequency=161, score=0.9355011401696003),
 KeywordScore(word=225, frequency=312, score=0.9322509173994322),
 KeywordScore(word=25676, frequency=111, score=0.9174517473486796),
 KeywordScore(word=2699, frequency=112, score=0.9167241129862725),
 KeywordScore(word=20888, frequency=249, score=0.9136806104549573),
 KeywordScore(word=19654, frequency=102, score=0.9116628110365063),
 KeywordScore(word=5167, frequency=105, score=0.909153060234049),
 KeywordScore(word=15070, frequency=105, score=0.909153060234049)]

In [76]:
for keyword in keywords[:10]:

    word = index2word[keyword.word]
    frequency = keyword.frequency
    score = keyword.score

    print('word=%s, frequency=%d, score=%.3f' % (
        word, frequency, score))

word=합의, frequency=101, score=0.964
word=효력, frequency=108, score=0.958
word=남북, frequency=161, score=0.936
word=19, frequency=312, score=0.932
word=해당하는, frequency=111, score=0.917
word=감시, frequency=112, score=0.917
word=제주, frequency=249, score=0.914
word=재개, frequency=102, score=0.912
word=군사분계선, frequency=105, score=0.909
word=아동, frequency=105, score=0.909


keyword 추출

In [81]:
df_dropna

,docID,date,writerName,title,content,tag,title_content
0,1,20230214,머니S,"정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원",정부가 미국 AI 개발업체인 '오픈AI'(OpenAI)의 '챗GPT'(ChatGPT...,0,"정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원 정부가 미..."
1,2,20230215,뉴시스,인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각,"기사내용 요약 법원 ""피의 사실 일부 다툼 여지, 불구속 상태 방어권 보장 필요"" ...",0,"인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각 기사내용 요약 법원 ""피의 사..."
2,3,20230214,아이뉴스24,튀르키예 강진에 우리나라 지하수가 출렁였다,튀르키예에서 발생한 강진에 우리나라의 지하수가 출렁였다는 관측 보고가 나왔다. 한국...,0,튀르키예 강진에 우리나라 지하수가 출렁였다 튀르키예에서 발생한 강진에 우리나라의 지...
3,4,20230215,데일리안,"멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행",[데일리안 = 박영민 기자] 오픈마켓 멸치쇼핑이 2023년 신입 및 경력 사원을 대...,0,"멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행 [데일리안 = 박영민 기..."
4,5,20230111,뉴스1,"美국방부, 추모의 벽 전사자 명단 오류에 ""유감스러운 실수""","국방부 대변인 ""실수 바로잡기 위해 내무부와 협력""…'오류 발견' 가족 등에 연락 ...",1,"美국방부, 추모의 벽 전사자 명단 오류에 ""유감스러운 실수"" 국방부 대변인 ""실수 ..."
...,...,...,...,...,...,...,...
7995,7996,20230104,이데일리,"외교부, 오는 12일 日강제징용 해법 토론회 연다",국회서 한일의원연맹과 공동 개최 [이데일리 권오석 기자] 외교부가 오는 12일 일제...,1,"외교부, 오는 12일 日강제징용 해법 토론회 연다 국회서 한일의원연맹과 공동 개최 ..."
7996,7997,20230215,아시아경제,"교원단체, '유치원' 명칭은 일제 잔재…'유아학교'로 변경해야",교육부가 2025년부터 유치원과 어린이집을 하나로 통합하겠다는 계획을 밝힌 가운데 ...,0,"교원단체, '유치원' 명칭은 일제 잔재…'유아학교'로 변경해야 교육부가 2025년부..."
7997,7998,20230212,뉴스1,내일 한일외교차관회담… '강제동원 해법' 이견 좁힐까,13일 워싱턴 한미일 차관협의회 계기 양자회담 예정 日 '사죄·배상' 방식 쟁점… ...,1,내일 한일외교차관회담… '강제동원 해법' 이견 좁힐까 13일 워싱턴 한미일 차관협의...
7998,7999,20230214,매일경제,최상호 국립오페라단 단장,문화체육관광부가 재단법인 국립오페라단 단장 겸 예술감독에 최상호 한국예술종합학교 음...,0,최상호 국립오페라단 단장 문화체육관광부가 재단법인 국립오페라단 단장 겸 예술감독에 ...


In [90]:
from soykeyword.proportion import CorpusbasedKeywordExtractor

macab = Mecab()
corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=20,
    min_df=2,
    tokenize=lambda x: mecab.nouns(x),
    verbose=True
)

# docs: list of str like
corpusbased_extractor.train(Corpus(df_dropna))

training was done 25128 terms, 7898 docs, memory = 0.619 Gb


In [93]:
documents = list(df_dropna.index)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,


In [117]:
keywords = corpusbased_extractor.extract_from_docs(
    [0],
    min_score=0,
    min_frequency=100
)
keywords[:10]

[KeywordScore(word='고급', frequency=113, score=0.9935620077255907),
 KeywordScore(word='대학원', frequency=255, score=0.9927271252151438),
 KeywordScore(word='신입', frequency=102, score=0.9883455262975699),
 KeywordScore(word='주권', frequency=104, score=0.9881174869061627),
 KeywordScore(word='소프트웨어', frequency=335, score=0.9872443098864283),
 KeywordScore(word='설계', frequency=428, score=0.9837293170907253),
 KeywordScore(word='반도체', frequency=1226, score=0.9825350303127635),
 KeywordScore(word='공고', frequency=164, score=0.9813249008959465),
 KeywordScore(word='실전', frequency=165, score=0.9812124822435045),
 KeywordScore(word='창출', frequency=189, score=0.9785221404743407)]

In [123]:
keywords_list = []
for i in tqdm(documents):
    keywordscore = corpusbased_extractor.extract_from_docs(
        [i],
        min_score=0.7,
        min_frequency=100
    )
    keywords_nbr = min(10, len(keywordscore))
    keywords = [(x.word, x.score) for x in keywordscore if len(x.word)>=2][:keywords_nbr]
    keywords_list.append(keywords)

100%|██████████| 7898/7898 [01:26<00:00, 91.81it/s]


In [135]:
df_dropna['keywords'] = keywords_list

C:\Users\vaiv\AppData\Local\Temp\ipykernel_7660\1657053099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna['keywords'] = keywords_list


In [141]:
len([x for x in keywords_list if len(x)==0])

148

In [142]:
result = pd.merge(df, df_dropna[['docID', 'keywords']], on='docID', how='left')

In [148]:
result.to_csv('../data/train_result.csv', index=False, encoding='utf-8-sig')